# pytorch로 Rnn 구현하기

In [1]:
import torch
import torch.nn as nn

In [2]:
input_size = 5
hidden_size = 8

In [3]:
inputs = torch.Tensor(1, 10, 5)

In [6]:
cell = nn.RNN(input_size, hidden_size, batch_first=True)
outputs, _status = cell(inputs)

In [8]:
print(outputs.shape)
print(_status.shape)

torch.Size([1, 10, 8])
torch.Size([1, 1, 8])


In [11]:
inputs = torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size = 5, hidden_size= 8, num_layers=2, batch_first=True)
# num_layers : 깊은 순환 신경망에서 층 쌓기
outputs, _status = cell(inputs)
print(outputs.shape)
print(_status.shape) # status 1 - > 2로 바뀜

torch.Size([1, 10, 8])
torch.Size([2, 1, 8])


In [13]:
inputs =torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size=5, hidden_size=8, num_layers=2, batch_first=True, bidirectional=True)
outputs, _status = cell(inputs)

print(outputs.shape) # batch size, sequence length, hidden size  * 2 // 단방향 RNN에서 은닉 크기가 2배
# 은닉상태 값이 연결되어서(concatenate)
print(_status.shape)
# 두번째 리턴값의 크기는 (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)

torch.Size([1, 10, 16])
torch.Size([4, 1, 8])


---

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [15]:
input_str = 'apple'
label_str = 'pple!'

char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)

print(f'문자 집합 크기 : {vocab_size}')

문자 집합 크기 : 5


In [35]:
input_size = vocab_size
hidden_size=  5
output_size = 5
lr = 0.1

In [17]:
char_to_idx = dict((c, i) for i,c in enumerate(char_vocab))
print(char_to_idx)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [18]:
idx_to_char={}
for key, val in char_to_idx.items() :
    idx_to_char[val] = key
print(idx_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [19]:
X_data = [char_to_idx[c] for c in input_str]
y_data = [char_to_idx[c] for c in label_str]

In [20]:
print(X_data ,y_data)

[1, 4, 4, 3, 2] [4, 4, 3, 2, 0]


In [21]:
# 배치 차원 추가
X_data = [X_data]
y_data = [y_data]
print(X_data, y_data)

[[1, 4, 4, 3, 2]] [[4, 4, 3, 2, 0]]


In [28]:
# one-hot vector
import numpy as np
X_one_hot = [np.eye(vocab_size)[x] for x in X_data] # np.eye() : 2-D array를 반환
print(X_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [30]:
X = torch.FloatTensor(X_one_hot)
y = torch.LongTensor(y_data)

C:\Users\PC\anaconda3\envs\yeseul\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [31]:
print(f'훈련 데이터 크기 : {X.shape}')
print(f'레이블 크기 : {y.shape}')

훈련 데이터 크기 : torch.Size([1, 5, 5])
레이블 크기 : torch.Size([1, 5])


### RNN 구현하기

In [38]:
class Net(nn.Module) :
    def __init__(self, input_size, hidden_size, output_size) :
        super(Net, self).__init__() 
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층
        
    def forward(self, x) :
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
    
net = Net(input_size, hidden_size, output_size)
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


In [39]:
# 배치 차원, 시점(timesteps), 출력 크기
print(outputs.view(-1, input_size).shape)
print(y.shape)
print(y.view(-1).shape)

torch.Size([5, 5])
torch.Size([1, 5])
torch.Size([5])


In [40]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr)

In [43]:
for i in range(100) :
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), y.view(-1))
    loss.backward() # 기울기 계산
    optimizer.step() # 파라미터 업데이트
    
    result = outputs.data.numpy().argmax(axis=2)
    # 최종 예측값인 time-step 별 5차원 벡터에 대해 가장 높은값의 인덱스를 선택
    result_str = ''.join([idx_to_char[c] for c in np.squeeze(result)])
    print(i, f'loss : {loss.item()}, prediction : {result}, true Y : {y_data}, prediction str : {result_str}')

0 loss : 1.507507562637329, prediction : [[3 3 0 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction str : ll!e!
1 loss : 1.3179529905319214, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction str : pppe!
2 loss : 1.162488579750061, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction str : pppe!
3 loss : 0.9838333129882812, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction str : pppe!
4 loss : 0.8142949342727661, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction str : pppe!
5 loss : 0.6452921032905579, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction str : pppe!
6 loss : 0.5141140818595886, prediction : [[4 4 4 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction str : pppe!
7 loss : 0.4067509174346924, prediction : [[4 4 3 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction str : pple!
8 loss : 0.33023741841316223, prediction : [[4 4 3 2 0]], true Y : [[4, 4, 3, 2, 0]], prediction str : pple!
9 loss : 0.264085590839386, p

---

# 문자 단위 RNN

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

charset = list(set(sentence))
char_dic = {c: i for i, c in enumerate(charset)}
print(char_dic)

{'a': 0, ' ': 1, 'g': 2, 'o': 3, 'l': 4, 'w': 5, 'm': 6, 's': 7, 'b': 8, 't': 9, 'i': 10, '.': 11, 'f': 12, 'r': 13, 'n': 14, 'h': 15, 'p': 16, 'e': 17, 'y': 18, ',': 19, 'k': 20, "'": 21, 'u': 22, 'c': 23, 'd': 24}


In [45]:
dic_size = len(char_dic)
print(f'문자 집합의 크기 : {dic_size}')

문자 집합의 크기 : 25


In [46]:
# 하이퍼 파라미터 설정
hidden_size = dic_size # 다른 값 줘도 무방
sequence_length = 10 # 샘플을 10개 단위로 끊어서 샘플을 만듦
lr = 0.1

In [47]:
# 데이터 구성하기
X_data =[]
y_data = []

for i in range(len(sentence) - sequence_length) :
    X_str = sentence[i : i+sequence_length]
    y_str = sentence[i + 1 : i+sequence_length + 1]
    print(i, X_str, '--->', y_str)
    
    X_data.append([char_dic[c] for c in X_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan ---> f you want
1 f you want --->  you want 
2  you want  ---> you want t
3 you want t ---> ou want to
4 ou want to ---> u want to 
5 u want to  --->  want to b
6  want to b ---> want to bu
7 want to bu ---> ant to bui
8 ant to bui ---> nt to buil
9 nt to buil ---> t to build
10 t to build --->  to build 
11  to build  ---> to build a
12 to build a ---> o build a 
13 o build a  --->  build a s
14  build a s ---> build a sh
15 build a sh ---> uild a shi
16 uild a shi ---> ild a ship
17 ild a ship ---> ld a ship,
18 ld a ship, ---> d a ship, 
19 d a ship,  --->  a ship, d
20  a ship, d ---> a ship, do
21 a ship, do --->  ship, don
22  ship, don ---> ship, don'
23 ship, don' ---> hip, don't
24 hip, don't ---> ip, don't 
25 ip, don't  ---> p, don't d
26 p, don't d ---> , don't dr
27 , don't dr --->  don't dru
28  don't dru ---> don't drum
29 don't drum ---> on't drum 
30 on't drum  ---> n't drum u
31 n't drum u ---> 't drum up
32 't drum up ---> t drum up 
33 t drum up  --->  

In [48]:
print(X_data[0] , y_data[0])

[10, 12, 1, 18, 3, 22, 1, 5, 0, 14] [12, 1, 18, 3, 22, 1, 5, 0, 14, 9]


In [50]:
X_one_hot = [np.eye(dic_size)[x] for x in X_data]
X = torch.FloatTensor(X_one_hot)
y = torch.LongTensor(y_data)

print(f'훈련 데이터의 크기 : {X.shape}, 레이블의 크기 : {y.shape}')

훈련 데이터의 크기 : torch.Size([170, 10, 25]), 레이블의 크기 : torch.Size([170, 10])


In [52]:
print(X[0]) # 원핫인코딩 결과
print(y[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [56]:
class Net(nn.Module) :
    def __init__(self, input_dim, hidden_dim, layers) :
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
        
    def forward(self, x) :
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net =Net(dic_size, hidden_size, 2)

In [54]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr)

In [57]:
outputs = net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


In [58]:
print(outputs.view(-1, dic_size).shape) # 이차원 텐서로 변환

torch.Size([1700, 25])


In [59]:
print(y.shape, y.view(-1).shape) # 정확도를 측정할 때는 펼쳐서 계산함

torch.Size([170, 10]) torch.Size([1700])


In [63]:
for i in range(100) :
    optimizer.zero_grad()
    outputs= net(X)
    loss = criterion(outputs.view(-1, dic_size), y.view(-1))
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(dim=2)
    pred_str = ''
    for j, result in enumerate(results) :
        if j ==  0 : # 처음엔 예측결과를 전부 가져옴
            pred_str += ''.join([charset[t] for t in result])
        else : # 그 이후로는 마지막 글자만 반복해서 추가함
            pred_str += charset[result[-1]]

    print(pred_str)
            

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

...

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffifffffffffff
fffifffffiffffffffkfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

---
# 단어 단위 RNN / embedding 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

sentence = 'Repeat is the best medicine for memory'.split()

In [2]:
vocab = list(set(sentence))

print(vocab)

['best', 'medicine', 'Repeat', 'memory', 'the', 'is', 'for']


In [5]:
word2idx = {tkn : i for i, tkn in enumerate(vocab, 1)}
word2idx['<unk>'] = 0 # UNK token

print(word2idx)

{'best': 1, 'medicine': 2, 'Repeat': 3, 'memory': 4, 'the': 5, 'is': 6, 'for': 7, '<unk>': 0}


In [7]:
idx2word = {v : k for k, v in word2idx.items()}
print(idx2word)
print(idx2word[1])

{1: 'best', 2: 'medicine', 3: 'Repeat', 4: 'memory', 5: 'the', 6: 'is', 7: 'for', 0: '<unk>'}
best


In [8]:
def build_data(sentence, word2idx) : 
    encoded = [word2idx[token] for token in sentence]
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력, 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가    
    return input_seq, label_seq

X, y = build_data(sentence, word2idx)
print(X, y)

tensor([[3, 6, 5, 1, 2, 7]]) tensor([[6, 5, 1, 2, 7, 4]])


In [10]:
class Net(nn.Module) :
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True) :
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size,
                                           embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size,
                               batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x) :
        # 1. Embedding layer
        output = self.embedding_layer(x)
        
        # 2. RNN layer
        output, hidden = self.rnn_layer(output)
        
        # 3. 출력층
        output = self.linear(output)
        
        return output.view(-1, output.size(2))
        
        
# hyperparams

vocab_size = len(word2idx)
input_size= 5
hidden_size = 20
        
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters())

output = model(X)
print(output) # 임의로 예측 

tensor([[ 0.1685,  0.1113, -0.0384,  0.1478, -0.1304, -0.0556,  0.0832,  0.0596],
        [ 0.3311,  0.1826,  0.0815,  0.0407,  0.4507, -0.2156,  0.0249,  0.0838],
        [ 0.3703,  0.0604, -0.2131,  0.1182, -0.1504, -0.0791,  0.2037, -0.0019],
        [ 0.3666,  0.1281, -0.2620, -0.0315,  0.5066, -0.1622,  0.1156,  0.3217],
        [ 0.1491, -0.1236, -0.5582, -0.0605,  0.6339, -0.4054,  0.1230,  0.3708],
        [ 0.3425,  0.0064, -0.1807, -0.1761,  0.7894, -0.5148,  0.1280,  0.0960]],
       grad_fn=<ViewBackward0>)


In [11]:
print(output.shape)

torch.Size([6, 8])


In [13]:
decode = lambda y : [idx2word.get(x) for x in y] # 디코딩(수치->단어)

In [19]:
for step in range(201) :
    optimizer.zero_grad() # 경사 초기화
    output = model(X) # 순방향 전파
    loss = loss_function(output, y.view(-1))
    loss.backward()
    optimizer.step()
    
    if step%40==0 :
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(' '.join(['Repeat'] + decode(pred)))
        print()

[01/201] 0.0310 
Repeat is the best medicine for memory

[41/201] 0.0258 
Repeat is the best medicine for memory

[81/201] 0.0219 
Repeat is the best medicine for memory

[121/201] 0.0189 
Repeat is the best medicine for memory

[161/201] 0.0164 
Repeat is the best medicine for memory

[201/201] 0.0145 
Repeat is the best medicine for memory

